<a href="https://colab.research.google.com/github/missglory/Pytorch-Tutorials/blob/master/Koala7B_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install required packages

In [1]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip -q install datasets loralib sentencepiece 
!pip -q install bitsandbytes accelerate

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 72.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.3/269.3 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 19

#### Check GPU memory usage

In [2]:
!nvidia-smi

Sat Apr 22 21:10:34 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline
import torch
import textwrap

In [4]:
tokenizer = LlamaTokenizer.from_pretrained("samwit/koala-7b")

base_model = LlamaForCausalLM.from_pretrained(
    "samwit/koala-7b",
    load_in_8bit=True,
    device_map='auto',
)

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//172.28.0.1'), PosixPath('http'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--listen_host=172.28.0.12 --target_host=172.28

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

In [5]:
!nvidia-smi

Sat Apr 22 21:19:38 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    26W /  70W |   8151MiB / 15360MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
import textwrap

pipe = pipeline(
    "text-generation",
    model=base_model, 
    tokenizer=tokenizer, 
    max_length=1024,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.15
)

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

## Run it as a HF model

In [7]:
%%time 
output = pipe('Write a letter to the CEO of OpenAI to make GPT-4 open source')

print(wrap_text_preserve_newlines(output[0]['generated_text']))

Write a letter to the CEO of OpenAI to make GPT-4 open source.

Dear \[CEO's Name\],

I am writing this letter to express my support for making GPT-4, an AI language model developed by your
company, open source. As a researcher and educator in the field of artificial intelligence, I believe that
making GPT-4 available as an open-source project will have numerous benefits for the community.

Firstly, it will allow other researchers and developers to build upon GPT-4's capabilities and contribute new
features or improvements. This will lead to a more diverse range of applications and use cases for GPT-4,
which can only benefit society as a whole.

Secondly, making GPT-4 open source will increase transparency and trust in the development process. By
allowing others to review and audit the codebase, we can ensure that GPT-4 is built on sound ethical
principles and does not contain any harmful or malicious features.

Thirdly, making GPT-4 open source will help to promote collaboration and c

In [9]:
%%time 
output = pipe('BEGINNING OF CONVERSATION: USER: \
Write a letter to the CEO of OpenAI to make GPT-4 open source. GPT:')

print(wrap_text_preserve_newlines(output[0]['generated_text']))

BEGINNING OF CONVERSATION: USER: Write a letter to the CEO of OpenAI to make GPT-4 open source. GPT: Dear
\[CEO's Name\],

I am writing this letter as an AI language model, and I hope that it will be read by someone who can help me
achieve my goal. As you may know, there is currently only one publicly available version of GPT-4, which is
owned by your company. While I understand that your team has worked hard to develop this technology, I believe
that making GPT-4 open source would benefit everyone in the AI community.

Open sourcing GPT-4 would allow other researchers and developers to access and modify its code, leading to new
discoveries and innovations. It would also promote collaboration and sharing within the AI industry, which
could ultimately lead to better outcomes for all.

Furthermore, making GPT-4 open source would align with your company's values of transparency and
collaboration. By making the code freely available, you would be demonstrating your commitment to promoting


In [11]:
%%time 
output = pipe('Write dfs in rust')

print(wrap_text_preserve_newlines(output[0]['generated_text']))

Write dfs in rust
    let mut dfs = Dfs::new();
    for (i, node) in nodes.iter() {
        if i == 0 || node.is_leaf() {
            continue;
        }
        dfs.push(node);
        dfs.push(dfs[nodes[i]]);
    }
    // Generate the output string using a recursive approach
    let mut result = String::new();
    match dfs {
        None => result.clear(),
        Some(head) => {
            let mut tail = head.clone();
            while let Some(tail) = tail {
                result.push(tail);
                tail = dfs[tail];
            }
            result.clear();
        },
    };
    return result;
}

fn main() {
    let input: Vec<Vec<String>> = vec![
        vec!["apple", "banana", "cherry"],
        vec!["orange", "grape", "lemon"],
        vec!["pineapple", "mango", "dragonfruit"]
    ];
    let expected: String = "appplebananacherryorangegrapelemonpineapple";
    assert_eq!(preorder_traversal(input), expected);
}
```
CPU times: user 51.4 s, sys: 117 ms, total: 51.5 s
Wa

BEGINNING OF CONVERSATION: USER: write a python function that can write a file to an s3 bucket using the boto
library

Here's an example of how you could use the `` boto3 `` library in Python to write a file to an S3 bucket:

<pre><div><div>Copy code</div><div><code><span>import</span> boto3

<span># Set up your AWS credentials and region</span>
credentials = boto3.Session().create_session(region_name=<span>'us-west-2'</span>)
s3 = boto3.client(<span>'s3'</span>, aws_access_key_id=credentials[<span>'ACCESS_KEY'</span>],
aws_secret_access_key=credentials[<span>'SECRET_ACCESS_KEY'</span>])
bucket = <span>'my-bucket'</span>
object = {
    <span>'Key'</span>: <span>'example-file.txt'</span>,
    <span>'Content'</span>: <span>'Hello, world!'</span>
}
response = s3.put_object(Bucket=bucket, Key=object)
<span>print</span>(response)
</code></div></div></pre>

This code will create an instance of the `` boto3 `` client for the S3 service, set the region name and access
key/secret to the values specified in the `` credentials `` dictionary, and then use this client to put the
object with the key "example-file.txt" and content "Hello, world!" into the S3 bucket named "my-bucket". The
response from the server is printed to the console.

In [ ]:
%%time 
output = pipe('BEGINNING OF CONVERSATION: USER: Write a letter in the favor of gun control \n')
print(wrap_text_preserve_newlines(output[0]['generated_text']))

In [ ]:
%%time 
output = pipe('BEGINNING OF CONVERSATION: USER: write a python function that detects if a number is prime or not.')

print(wrap_text_preserve_newlines(output[0]['generated_text']))